---
title: "Descriptive Statistics"
author: Josie Athens
date: today
highlight-style: nord
format:
  html:
    code-fold: false
    theme: mint
    toc: true
    number-sections: true
    toc-title: Contents
    smooth-scroll: true
    embed-resources: true
execute: 
  cache: true
  warning: false
---

# [📖 Main Menu](index.html)

In [1]:
#| output: false
library(tidyverse)
library(rstatix)
library(easystats)
library(pubh)
library(sjlabelled)

options('huxtable.knit_print_df' = FALSE)
options('huxtable.autoformat_number_format' = list(numeric = "%5.2f"))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘rstatix’


The following object is masked from ‘package:stats’:

    filter


# Attaching packages: easystats 0.7.2 (red = needs update)
✔ bayestestR  0.13.2   ✔ correlation 0.8.5 
✔ datawizard  0.11.0   ✖ effectsize  0.8.8 
✔ insight     0.20.1   ✔ modelbased  0.8.8 
✔ performance 0.12.0   ✔ parameters  0.22.0
✔ report      0.5.8    ✔ see         0.8.4 

Restart the R-Session and update packages with `easystats::easystats_update()`.


Loading require

# Overview

In this laboratory, we will look at performing descriptive statistics and tabular analysis. We will learn to make quality tables and how to export them for publication.

Once you have completed this lab, you should feel comfortable:

-   Obtaining basic descriptive statistics for categorical variables (factors).
-   Obtaining basic descriptive statistics for continuous variables.
-   Making tables of descriptive statistics.
-   Exporting tables and inserting them into other applications.

**Summary of New Commands**

| **Command**      | **Library** | **Function**                                                                      |
|------------------|-------------|-----------------------------------------------------------------------------------|
| **as_hux**       | *huxtable*  | Converts a objects into a hux tables                                              |
| **add_footnote** | *huxtable*  | Adds a footnote to a hux table                                                    |
| **cosm_sum**     | *pubh*      | Cosmetics for *tbl_summary*                                                       |
| **cross_tbl**    | *pubh*      | Tabulates contingency tables                                                      |
| **descr**        | *sjmisc*    | Descriptive statistics of continuous variables                                    |
| **estat**        | *pubh*      | Descriptive statistics of continuous variables                                    |
| **freq_cont**    | *pubh*      | Frequency tables of continuous variables                                          |
| **group_by**     | *dplyr*     | Group by one or more variables                                                    |
| **insert_row**   | *huxtable*  | Adds rows on top of hux tables                                                    |
| **median**       | *base*      | Median                                                                            |
| **n**            | *dplyr*     | Number of observations                                                            |
| **rec**          | *sjmisc*    | Recode variables                                                                  |
| **rel_dis**      | *pubh*      | Coefficient of variation                                                          |
| **sd**           | *base*      | Standard deviation                                                                |
| **seq**          | *base*      | Creates a vector of regular sequences                                             |
| **summarise**    | *dplyr*     | Summarise continuous variables by given statistics                                |
| **summary**      | *base*      | Summary of objects                                                                |
| **tbl_summary**  | *gtsummary* | Constructs tables of descriptive statistics                                       |
| **tbl_strata**   | *gtsummary* | Constructs tables of descriptive statistics stratified by 2 categorical variables |
| **theme_pubh**   | *pubh*      | Theme for hux tables                                                              |

# Introduction

For understanding descriptive statistics, first we should be able to distinguish between:

1.  Big and small datasets. For our paper, we will consider a large data set if the number of observations is greater or equal than 100. We will define a small dataset if the number of observations is less or equal than 30.
2.  Numerical and categorical variables.
3.  Continuous and discrete variables.
4.  Continuous variables with a Normal distribution (or close enough) and those not normally distributed.

::: callout-note

We will use two datasets: `wcgs` as an example of a large dataset and `kfm` as an example of a small dataset.

:::

::: callout-caution
## Exercise

Load the `wcgs.rds` dataset we created in the previous lab.

:::

In [2]:
#| code-fold: true
wcgs = read_rds('data/wcgs.rds')

We will also use the `kfm` dataset from the `ISwR` package; the data records information about infants weighed before and after breast feeding. For more information, you can look at the corresponding help file. First, we load the data.

In [3]:
data(kfm, package = "ISwR")
kfm = kfm |> as_tibble()

kfm |> head()

no,dl.milk,sex,weight,ml.suppl,mat.weight,mat.height
<int>,<dbl>,<fct>,<dbl>,<int>,<int>,<int>
1,8.42,boy,5.002,250,65,173
4,8.44,boy,5.128,0,48,158
5,8.41,boy,5.445,40,62,160
10,9.65,boy,5.106,60,55,162
12,6.44,boy,5.196,240,58,170
16,6.29,boy,5.526,0,56,153


Variable `sex` is categorical, a factor with two levels or categories, let's take a look at them:

In [4]:
kfm %$% levels(sex)

[1] "boy"  "girl"

I would like the levels of `sex` to have the first letter in capitals (this is optional). There are different ways of changing names of levels in a factor, one of them is to simply provide a vector of characters with the new names, in the same order as they appear in the levels of the variable. A second option is to use `recode_values`. In `recode_vales` we provide new values in a `list`, so the order does not really matter. The first argument, at the left side of  `=` is the new value or new name, while the second argument, is the old value.

In [5]:
kfm = kfm %>%
  mutate(
    sex = recode_values(sex,  list (Girl = "girl",  Boy = "boy"))
  )

kfm %$% levels(sex)

[1] "Boy"  "Girl"

We add labels to `kfm` and save it in our directory.

In [6]:
kfm = kfm |>
  var_labels(
     dl.milk = 'Breast-milk intake (dl/day)',
     sex = 'Sex',
     weight = 'Child weight (kg)',
     ml.suppl = 'Milk substitute (ml/day)',
     mat.weight = 'Maternal weight (kg)',
     mat.height = 'Maternal height (cm)'
     )

In [7]:
write_rds(kfm, "data/kfm.rds")

# Categorical variables

For categorical variables (including ordinal variables), we want to know and report the number of observations on each category. We can use `freq_table` from the `rstatix` package.

In [8]:
wcgs |> freq_table(arcus)

arcus,n,prop
<fct>,<int>,<dbl>
Absent,2211,70.1
Present,941,29.9


In [9]:
wcgs |> freq_table(arcus, na.rm = FALSE)

arcus,n,prop
<fct>,<int>,<dbl>
Absent,2211,70.1
Present,941,29.8
NA,2,0.1


## Epidemiology tables

In epidemiology, it is frequent to have two by two (2 $\times$ 2) tables. In those cases, the outcome is binary (e.g., disease: present/absent) and the exposure is binary too. Sometimes, it is also desirable to have $n \times$ 2 tables in which we are comparing more than two groups about a particular binary outcome. For *PUBH 725* and *PUBH 726*, we will display the exposure of interest in the columns so the percentages for each categorical variable represent prevalences.

In [10]:
#| label: tbl-tab1
#| tbl-cap: "Distribution of coronary heart disease events (CHD) by behaviour pattern."
#| output: asis
wcgs |>
  mutate(
    chd = relevel(chd, ref = "CHD"),
  ) |>
  copy_labels(wcgs) |>
  select(dib_pat, chd) |> 
  cross_tbl(by = "dib_pat") |>
  theme_pubh(2) |> 
  add_footnote("Data from the WCGS.", font_size = 9) |>
  print_html()

<table class="huxtable" data-quarto-disable-processing="true" style="border-collapse: collapse; border: 0px; margin-bottom: 2em; margin-top: 2em; ; margin-left: auto; margin-right: auto;  ">
<col><col><col><col><tr>
<th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold;"></th><th colspan="2" style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold;"><p><strong>Dichotomous behaviour</strong></p>
</th><th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold;"></th></tr>
<tr>
<th style="vertical-align: top; text-align: left; white-space: normal; border-style: solid solid solid solid; border-w

::: callout-note
1.  In the previous code, we changed the reference for our outcome as it is standard to report first cases/treatment and then controls.
2.  With `mutate` we are generating new variables, which happen to have the same names as the old variables, thus we replace them. By doing that, as they are new, they are no longer labelled. To preserve the labels, we use `copy_labels`.
3.  As we did not assign the changes to a new data frame, the changes we made were not stored and their impact would be only in the code we provided. Why is this important? Because if we perform inferential statistics, we want for controls to still be the reference.
4.  Formal tables, as those used for reports, should be self-explained, formatted, include units, use good labels, be numbered and include a caption/legend.
    1.  The first line in the code block, is hidden on reports, I used it for the label: `#| label: tbl-tab1`
    2.  The second line in the code block, is hidden on reports, I used it for the caption: `#| tbl-cap: "Distribution of coronary heart disease events (CHD) by behaviour pattern."`
    3.  We can add more information in footnotes with `add_footnote`. In the current example, the size of the font is smaller (9 pt).
5.  To reference the table in the text we type `@label` in our Markdown script, where `label` is the name of the chunk were we construct the table which starts with `tbl-`. In our current example, to refer to our first table, we would type: `@tbl-tab1`.
:::

By default, `cross_tbl` uses the label of the outcome as the name of the first column. When you `knit` to Word, the table is exported and you can edit it there, add captions, etc. Package `huxtable` allows us to add rows, caption, footnotes, etc.

::: callout-caution
##  Exercise
Generate a table for the relationship between behavioural pattern (`beh_pat`) and the presence of CHD (`chd`) from the WCGS dataset. Remember to display the exposure of interest in columns, so percentages represent prevalence.
:::

In [11]:
#| code-fold: true
#| label: tbl-tab2
#| tbl-cap: "Distribution of behavioural pattern by coronary heart disease (CHD) event."
#| output: asis
wcgs |>
  mutate(
    chd = relevel(chd, ref = "CHD")
  ) |>
  copy_labels(wcgs) |>
  select(beh_pat, chd) |> 
  cross_tbl(by = "beh_pat") |>
  theme_pubh(2) |> 
  add_footnote("Data from the WCGS.", font_size = 9) |>
  print_html()

<table class="huxtable" data-quarto-disable-processing="true" style="border-collapse: collapse; border: 0px; margin-bottom: 2em; margin-top: 2em; ; margin-left: auto; margin-right: auto;  ">
<col><col><col><col><col><col><tr>
<th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold;"></th><th colspan="4" style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold;"><p><strong>Behaviour pattern</strong></p>
</th><th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold;"></th></tr>
<tr>
<th style="vertical-align: top; text-align: left; white-space: normal; border-style: solid solid solid solid; bo

::: callout-warning
## Question

What is your main observation from the distribution of cases?
:::

::: callout-note
## Answer

The prevalence of CHD in males in the **A** groups (\~ 11%) is about the double than the prevalence in males in the **B** groups (\~ 5%). The prevalence of CHD in the population is 8.1%.
:::

## Double stratification

There are cases when we want to present distribution of categorical variables by a exposure of interest and stratified by another categorical variable (e.g. confounder) of interest.

The following code shows how to generate this kind of tables.

In [12]:
#| label: tbl-tab3
#| tbl-cap: "Distribution of coronary heart disease events (CHD) by behaviour pattern and smoking status."
#| output: asis
wcgs |> 
  select(chd, smoker, dib_pat) |> 
  mutate(
    chd = relevel(chd, ref = "CHD"),
    smoker = relevel(smoker, ref = "Smoker")
  ) |>
  copy_labels(wcgs) |>
  tbl_strata(
    strata = smoker,
    .tbl_fun = ~ .x |>
      tbl_summary(by = dib_pat, missing = "no")
  ) |> 
  cosm_sum() %>% theme_pubh(2) |> 
  set_align(1, everywhere, "center") |> 
  add_footnote("Data from the WCGS.", font_size = 9) |>
  print_html()

<table class="huxtable" data-quarto-disable-processing="true" style="border-collapse: collapse; border: 0px; margin-bottom: 2em; margin-top: 2em; ; margin-left: auto; margin-right: auto;  ">
<col><col><col><col><col><tr>
<th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold;"></th><th colspan="2" style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold;"><p><strong>Smoker</strong></p>
</th><th colspan="2" style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold;"><p><strong>Non-Smoker</strong></p>
</th></tr>
<tr>
<th style="vertical-align: top; text-align: left; white-space: normal; border-sty

::: callout-note
1.  We construct the table with `tbl_strata` from `gtsummary`.
2.  `tbl_summary` is the function behind `cross_tbl`.
3.  The argument `missing = "no"` is used to not shown missing values.
4.  With `cosm_sum` we add cosmetics to the table and converts it to a *huxtable*.
:::

# Continuous variables

For continuous variables, we want to report a measure of central tendency and a measure of dispersion. To inspect a continuous variable, we would like to know more than only two statistics. For those purposes, we *summarise* the data. The function `summary` is a very basic function that displays a summary of statistics for variables, or information about statistical models (we will look at this capability in PUBH 726).

There are several options to look at descriptive statistics of continuous variables. We will compare the following options:

1.  `descr` from `sjmisc`
2.  `estat` from `pubh`
3.  `summarise` from `dplyr`

Function `descr` has the advantage of including variable labels. It can be used to report the descriptive statistics of more than one variable. By default, it shows all the following columns:

| Column name | Description                                |
|-------------|--------------------------------------------|
| Variable       | Label of variable                          |
| Mean        | Arithmetic mean                            |
| SD          | Standard deviation                         |
| IQR | Interquartile range |
| Min | Minimum value |
| Max | Maximum value |
| Skewness | Skewness (distribution) |
| Kurtosis | Shape of the distribution |
| n           | Total number of observations |
| n_Missing      |  Number of missing values   |

For example, the descriptive statistics of blood cholesterol:

In [13]:
#| label: tbl-tab4
#| tbl-cap: "Descriptive statistics of blood cholesterol."
wcgs |>
  select(chol) |>
  describe_distribution()

Variable,Mean,SD,IQR,Min,Max,Skewness,Kurtosis,n,n_Missing
<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<int>
chol,226.3724,43.42043,56,103,645,0.6767978,3.038827,3142,12


::: callout-note
1.  First, we assign the table to a new object, `tab4`, so we can edit the name of the columns.
2.  Next, we generate a *hux* table.
3.  There are different *themes* available for *huxtables*. Be consistent and use the same theme for all your tables.
4.  The theme `theme_pubh` draws horizontal lines bellow a particular `row`. The argument is a scalar or a vector which define the rows on which one wants to draw an under line, which in this example, is row one.
:::

The descriptive statistics of all continuous variables in `wcgs`:

In [14]:
#| label: tabl-tab5
#| tbl-cap: "Descriptive statistics of continuous variables."
wcgs |>
  select(age, height, weight, sbp, dbp, chol) |>
  describe_distribution(iqr = FALSE)

Variable,Mean,SD,Min,Max,Skewness,Kurtosis,n,n_Missing
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
age,46.27869,5.524045,39.0000,59.000,0.5266160,-0.7719188,3154,0
height,177.23547,6.422881,152.4000,198.120,-0.1986184,0.2798079,3154,0
weight,77.09100,9.569037,35.3808,145.152,0.6034711,1.9361996,3154,0
sbp,128.63285,15.117731,98.0000,230.000,1.2049698,2.7988014,3154,0
dbp,82.01554,9.726880,58.0000,150.000,0.9283029,2.3692373,3154,0
chol,226.37237,43.420426,103.0000,645.000,0.6767978,3.0388272,3142,12


::: callout-note
1.  For illustration purposes, I used *theme_blue*.
2.  Be consistent and use the same theme for all your tables.
3.  The command `set_all_padding` is useful to change the space between the rows (padding).
:::

The function `estat` from the `pubh` package displays all the statistics that we are interested to calculate from continuous variables.

In [15]:
#| label: tbl-tab6
#| tbl-cap: "Descriptive statistics of blood cholesterol."
wcgs |> 
  estat(~ chol) |>
  as_hux() |> theme_pubh() |> set_width(0.8) |> 
  add_footnote("Data from the WCGS.", font_size = 9)

,,N,Min,Max,Mean,Median,SD,CV
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,,N,Min,Max,Mean,Median,SD,CV
1.1,Cholesterol (mg/dl),3142,103,645,226.37,223,43.42,0.19
,Data from the WCGS.,Data from the WCGS.,Data from the WCGS.,Data from the WCGS.,Data from the WCGS.,Data from the WCGS.,Data from the WCGS.,Data from the WCGS.


In `estat` `N` is the number of observations, `Min.` is the minimum value, `Max.` is the maximum value, `SD` is the standard deviation and `CV` is the coefficient of variation or *relative dispersion*, defined as:

$$CV = \frac{SD}{Mean}$$

Finally, we can also decide which statistics we want to report using `summarise`.

::: callout-tip
Read the help file of  `summarise` to know more about available functions.
:::

In [16]:
wcgs |>
  summarise(
    N = n(),
    Mean = mean(chol, na.rm = TRUE),
    SD = sd(chol, na.rm = TRUE),
    Median = median(chol, na.rm = TRUE),
    CV = rel_dis(chol)
  )

N,Mean,SD,Median,CV
<int>,<dbl>,<dbl>,<dbl>,<dbl>
3154,226.3724,43.42043,223,0.1918097


## Understanding dispersion

We will use the relative dispersion to gather information about the variability in our variable of interest. When we multiply the *CV* by 100, we have it as a percentage. Values of *CV* \< 10% indicate relatively small variability, which is better for further analysis. Values of *CV* \> 33% are of concern. We will consider variables with *CV* \> 30% having high variability and not normally distributed.

In the normal distribution, about three standard deviations to the right and the left, include 99% of the population. If we had a physiological variable with a *CV* = 40%, some of the observations would be negative, and that would not be possible unless the variable was a difference.

## Formulas

We frequently use formulas in `R`. The general syntax of a formula is:

`y ~ x, data = dataset`

Where:

-   `y` is the *response* variable (aka: dependent or outcome).
-   `x` is the *explanatory* variable (aka independent, predictor or exposure). We can have more than one explanatory variable (e.g., confounders).
-   `dataset` is the name of the data frame or tibble where the variables `x` and `y` can be found.

::: callout-important
The symbol `~` is used in `R` for *formulas*. It can be interpreted as *depends on*. In the most typical scenario: `y ~ x` means "`y` depends on `x`" or "`y` is a function of `x`".
:::

## Frequency tables for continuous variables

For relatively large samples, we can take a look at the distribution of a continuous outcome by breaking values in different number of classes (between 8 and 20) and constructing a frequency table:

In [17]:
wcgs %$%
  freq_cont(chol, bks = seq(100, 650, 50))

Class,Mids,Counts,rel.freq,cum.freq
<fct>,<dbl>,<int>,<dbl>,<dbl>
"(100,150]",125,89,0.03,0.03
"(150,200]",175,761,0.24,0.27
"(200,250]",225,1454,0.46,0.73
"(250,300]",275,673,0.21,0.94
"(300,350]",325,144,0.05,0.99
"(350,400]",375,19,0.01,1.00
"(400,450]",425,1,0.00,1.00
"(450,500]",475,0,0.00,1.00
"(500,550]",525,0,0.00,1.00


::: callout-note
We use `seq` to generate a sequence of numbers, the first argument is the starting point, the second argument the ending point and the third argument the size of the step.
:::

Frequency tables are the basis for constructing histograms and they represent a way to better understand the distribution of continuous variables.

## Interpreting statistics

Look at the minimum, maximum and the mean value of cholesterol. The mean is clearly closer to the minimum value than to the maximum, i.e., it is not in the centre of the two values. This fact indicates that we have subjects with relatively high values than the rest of the population; it may be just one *outlier*, a couple of *outliers* or that the distribution is clearly skewed to the right (positive).

We will always start our analysis with descriptive and graphical analysis, to understand our variables. That does not mean that we are going to report everything: we have to be selective.

In other words, for our preliminary analysis, when we are *inspecting* variables, we look at all the statistics and the distribution of continuous variables. For a manuscript, we would report only two statistics:

1.  For variables following a normal distribution, we report the mean and the standard deviation. In most cases, these statistics are reported even when the variable is not normally distributed.
2.  For variables which are clearly not normally distributed, we could report the median and the interquartile range (IQR).

## Stratified Analysis

It's common that we want to compare statistics from different groups, in the most typical scenario, the control and the experimental (or cases) group. For this reason, it is helpful to display statistics of continuous variables, stratified by groups of interest.

In [18]:
#| label: tbl-tab7
#| tbl-cap: "Descriptive statistics of blood cholesterol by coronary heart disease (CHD) event."
wcgs |>
  estat(~ chol|chd) |>
  as_hux() |> theme_pubh() |> set_width(0.8) |> 
  add_footnote("Data from the WCGS.", font_size = 9)

,,Coronary Heart Disease,N,Min,Max,Mean,Median,SD,CV
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,,Coronary Heart Disease,N,Min,Max,Mean,Median,SD,CV
1.1,Cholesterol (mg/dl),No CHD,2885,103,400,224.26,221,42.22,0.19
2,,CHD,257,155,645,250.07,245,49.4,0.2
,Data from the WCGS.,Data from the WCGS.,Data from the WCGS.,Data from the WCGS.,Data from the WCGS.,Data from the WCGS.,Data from the WCGS.,Data from the WCGS.,Data from the WCGS.


::: callout-note
It is valid to input the `formula` as either `chol ~ chd` (read as `chol` depends on `chd`) or as `~ chol|chd` (read as `chol` given `chd`).
:::

Alternatively, we can use `group_by`.

In [19]:
wcgs |>
  group_by(chd) |>
  select(chol) |>
  describe_distribution(iqr = FALSE)

Adding missing grouping variables: `chd`


Variable,Mean,SD,Min,Max,Skewness,Kurtosis,n,n_Missing,.group
<chr>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<chr>
chol,224.2614,42.21679,103,400,0.4133812,0.4787509,2885,12,chd=No CHD
chol,250.0700,49.39552,155,645,2.2547935,15.3592960,257,0,chd=CHD


In [20]:
wcgs |>
  group_by(chd) |>
  summarise(
    N = n(),
    Mean = mean(chol, na.rm = TRUE),
    SD = sd(chol, na.rm = TRUE),
    Median = median(chol, na.rm = TRUE),
    CV = rel_dis(chol)
  ) 

chd,N,Mean,SD,Median,CV
<fct>,<int>,<dbl>,<dbl>,<int>,<dbl>
No CHD,2897,224.2614,42.21679,221,0.1882482
CHD,257,250.0700,49.39552,245,0.1975267


::: callout-warning
## Question

What can you conclude from the descriptive statistics of cholesterol by CHD status?
:::

::: callout-note
## Answer

Subjects with a CHD event from the WCGS dataset have a mean value of blood cholesterol higher than those without a CHD event. We do not know if that difference is statistically significant or not.
:::

The advantage of `group_by` is that we can stratify for more than one variable, for example:

In [21]:
wcgs |>
  data_summary( 
    n = n(),
    Mean = mean(chol,  na.rm = TRUE), 
    SD = sd(chol,  na.rm = TRUE),
    CV = rel_dis(chol),
    by = c("chd", "smoker"),
    include_na = FALSE
  )

Warning message in mean.default(chol, na.rm = TRUE):
“argument is not numeric or logical: returning NA”
Warning message in mean.default(chol, na.rm = TRUE):
“argument is not numeric or logical: returning NA”
Warning message in mean.default(chol, na.rm = TRUE):
“argument is not numeric or logical: returning NA”
Warning message in mean.default(chol, na.rm = TRUE):
“argument is not numeric or logical: returning NA”


chd,smoker,n,Mean,SD,CV
<fct>,<fct>,<int>,<dbl>,<dbl>,<dbl>
No CHD,Non-Smoker,1554,221.0090,40.72507,0.1842688
No CHD,Smoker,1343,228.0322,43.59538,0.1911808
CHD,Non-Smoker,98,243.3776,56.86274,0.2336400
CHD,Smoker,159,254.1950,43.85585,0.1725284


In [22]:
#| label: tbl-tab8
#| tbl-cap: "Descriptive statistics of cholesterol by coronary heart disease (CHD) event and smoking status."
#| output: asis
wcgs |> 
  select(chol, chd, smoker) |> 
  mutate(
    chd = relevel(chd, ref = "CHD"),
    smoker = relevel(smoker, ref = "Smoker")
  ) |>
  copy_labels(wcgs) |>
  tbl_strata(
    strata = smoker,
    .tbl_fun = ~ .x |>
      tbl_summary(by = chd, missing = "no") 
  ) |> 
  cosm_sum() |> theme_pubh(2) |> 
  set_align(1, everywhere, "center") |> 
  add_footnote("Median (IQR). Data from the WCGS dataset.", 
               font_size = 9) |> print_html()

<table class="huxtable" data-quarto-disable-processing="true" style="border-collapse: collapse; border: 0px; margin-bottom: 2em; margin-top: 2em; ; margin-left: auto; margin-right: auto;  ">
<col><col><col><col><col><tr>
<th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold;"></th><th colspan="2" style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold;"><p><strong>Smoker</strong></p>
</th><th colspan="2" style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold;"><p><strong>Non-Smoker</strong></p>
</th></tr>
<tr>
<th style="vertical-align: top; text-align: left; white-space: normal; border-sty

::: callout-caution
## Exercise
Compare the statistics of breast-milk intake (`dl.milk`) by sex from the `kfm` dataset.
:::

In [23]:
#| code-fold: true
#| label: tbl-tab9
#| tbl-cap: "Descriptive statistics of breast-milk intake in babies."
kfm |>
  estat(dl.milk ~ sex) |>
  as_hux() |> theme_pubh() |>
  add_footnote("Data from the kfm dataset.", font_size = 9)

,,Sex,N,Min,Max,Mean,Median,SD,CV
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,,Sex,N,Min,Max,Mean,Median,SD,CV
1.1,Breast-milk intake (dl/day),Boy,25,4.91,10.43,7.95,8.39,1.49,0.19
2,,Girl,25,4.44,10.03,7.06,7.22,1.42,0.2
,Data from the kfm dataset.,Data from the kfm dataset.,Data from the kfm dataset.,Data from the kfm dataset.,Data from the kfm dataset.,Data from the kfm dataset.,Data from the kfm dataset.,Data from the kfm dataset.,Data from the kfm dataset.


::: callout-warning
## Question

What are your main observations?
::: 

::: callout-note
## Answer

Breast-milk intake by boys is almost one more dl/day than girls. We do not know if this difference is significant or not. Variability in breast-milk intake was the same between boys and girls.
:::

# Creating tables for publication

It is very common to report the descriptive statistics from the data in the first table of papers. The strategy is to calculate the descriptive statistics for all our variables and then export it in a way the table could be read by a word processor or spreadsheet application.

We can construct a single table of descriptive statistics for all of our variables with a single command. The function `mytable` from the `moonBook` package allows us to do that. For continuous variables, `mytable` displays means and standard deviations by default but that can be changed to display medians and inter quantile ranges instead.

We partially remove `id` and `ncigs` it from the dataset using *negative indexing*.

In [24]:
#| label: tbl-tab10
#| tbl-cap: "Descriptive statistics of the WCGS dataset."
#| output: asis
wcgs |> 
  select(- c(id, ncigs)) |> 
  tbl_summary() |> 
  cosm_sum(bold = TRUE) |> 
  add_footnote("Median (IQR); n (%)", font_size = 9) |>
  print_html()

<table class="huxtable" data-quarto-disable-processing="true" style="border-collapse: collapse; border: 0px; margin-bottom: 2em; margin-top: 2em; ; margin-left: auto; margin-right: auto;  ">
<col><col><tr>
<th style="vertical-align: top; text-align: left; white-space: normal; border-style: solid solid solid solid; border-width: 0pt 0pt 0.4pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: normal;"><p><strong>Variable</strong></p>
</th><th style="vertical-align: top; text-align: right; white-space: normal; border-style: solid solid solid solid; border-width: 0pt 0pt 0.4pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: normal;"><p><strong>N = 3,154</strong></p>
</th></tr>
<tr>
<td style="vertical-align: top; text-align: left; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold;">Age (years)</td><td style="vertical-align: top; text-align: right; white-space: normal; border-style: solid solid solid solid

## Stratified tables

We can use `cross_tbl` to report descriptive statistics of relevant variables, by exposure of interest. We will use negative selection, removing some columns.

In [25]:
#| label: tbl-tab11
#| tbl-cap: "Descriptive statistics of the WCGS dataset by behaviour pattern."
#| output: asis
wcgs |>
  select(-c(id, beh_pat, ncigs, time)) |>
  mutate(
    chd = relevel(chd, ref = "CHD"),
    arcus = relevel(arcus, ref = "Present"),
    smoker = relevel(smoker, ref = "Smoker")
  ) |>
  copy_labels(wcgs) |>
  cross_tbl(by = "dib_pat") |> 
  add_footnote("n (%); Median (IQR)", font_size = 9) |>
  print_html()

<table class="huxtable" data-quarto-disable-processing="true" style="border-collapse: collapse; border: 0px; margin-bottom: 2em; margin-top: 2em; ; margin-left: auto; margin-right: auto;  ">
<col><col><col><col><tr>
<th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: normal;"></th><th colspan="2" style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: normal;"><p><strong>Dichotomous behaviour</strong></p>
</th><th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: normal;"></th></tr>
<tr>
<th style="vertical-align: top; text-align: left; white-space: normal; border-style: solid solid solid solid; border-w

The WCGS was a cohort study, hence, observational. Some people like to display a column with $p$-values comparing the groups; `cross_tab` can display $p$-values for stratified tables. If displayed, those $p$-values come from univariate analysis.

In [26]:
#| label: tbl-tab12
#| tbl-cap: "Descriptive statistics of the WCGS dataset by behaviour pattern."
#| output: asis
wcgs |>
  select(-c(id, beh_pat, ncigs, time)) |>
  mutate(
    chd = relevel(chd, ref = "CHD"),
    arcus = relevel(arcus, ref = "Present"),
    smoker = relevel(smoker, ref = "Smoker")
  ) |>
  copy_labels(wcgs) |>
  cross_tbl(by = "dib_pat", p_val = TRUE) |>  
  add_footnote("n(%); Median (IQR);",font_size = 9) |>
  print_html()

<table class="huxtable" data-quarto-disable-processing="true" style="border-collapse: collapse; border: 0px; margin-bottom: 2em; margin-top: 2em; ; margin-left: auto; margin-right: auto;  ">
<col><col><col><col><col><tr>
<th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: normal;"></th><th colspan="2" style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: normal;"><p><strong>Dichotomous behaviour</strong></p>
</th><th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: normal;"></th><th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width

In all tables before, for continues variables, `cross_tbl` reports by default median and IQR. By including the argument `method = 1`, `cross_tbl` would report mean and standard deviations.

In [27]:
#| label: tbl-tab13
#| tbl-cap: "Descriptive statistics of the WCGS dataset by behaviour pattern."
#| output: asis
wcgs |>
  select(-c(id, beh_pat, ncigs, time)) |>
  mutate(
    chd = relevel(chd, ref = "CHD"),
    arcus = relevel(arcus, ref = "Present"),
    smoker = relevel(smoker, ref = "Smoker")
  ) |>
  copy_labels(wcgs) |>
  cross_tbl(by = "dib_pat", method = 1) |> 
  set_width(0.9) |>  
  add_footnote("n (%); Mean (SD)", font_size = 9) |>
  print_html()

<table class="huxtable" data-quarto-disable-processing="true" style="border-collapse: collapse; border: 0px; margin-bottom: 2em; margin-top: 2em; width: 90%; margin-left: auto; margin-right: auto;  ">
<col><col><col><col><tr>
<th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: normal;"></th><th colspan="2" style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: normal;"><p><strong>Dichotomous behaviour</strong></p>
</th><th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: normal;"></th></tr>
<tr>
<th style="vertical-align: top; text-align: left; white-space: normal; border-style: solid solid solid solid